---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [60]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [16]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    f=open("university_towns.txt")
    data=[x.rstrip() for x in f]
    
    
    state_town=[]
    
    for i in data:
        if i[-6:]=='[edit]':
            state=i[:-6]
        elif '(' in i:
            towns=i[:i.index('(')-1]
            state_town.append([state,towns])
            
        else:
            towns=i
            state_town.append([state,towns])
            
    df=pd.DataFrame(state_town,columns=['State','RegionName'])
            
    
    
    return df
#get_list_of_university_towns()







In [20]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    gdp=pd.read_excel('gdplev.xls',skiprows=7)
    
    gdp=gdp[['Unnamed: 4','Unnamed: 5']]
    gdp=gdp[212:]
    quarters=[]
    gdp.columns=['Quarter','GDP']
    
    gdp['GDP']=pd.to_numeric(gdp['GDP'])
    for i in range(len(gdp)-2):
        if (gdp.iloc[i][1]>gdp.iloc[i+1][1]) and (gdp.iloc[i+1][1]>gdp.iloc[i+2][1]) :
            quarters.append(gdp.iloc[i][0])
    
    return quarters[0]


#get_recession_start()

'2008q3'

In [19]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    gdp=pd.read_excel('gdplev.xls',skiprows=7)
    
    gdp=gdp[['Unnamed: 4','Unnamed: 5']]
    gdp=gdp[212:]
    quarters=[]
    gdp.columns=['Quarter','GDP']
    
    gdp['GDP']=pd.to_numeric(gdp['GDP'])
    for i in range(len(gdp)-4):
        if (gdp.iloc[i][1]>gdp.iloc[i+1][1]) and (gdp.iloc[i+1][1]>gdp.iloc[i+2][1]) and (gdp.iloc[i+3][1]>gdp.iloc[i+2][1]) and (gdp.iloc[i+4][1]>gdp.iloc[i+3][1]):
            quarters.append(gdp.iloc[i+4][0])    
   
    return quarters[0]
#get_recession_end()

'2009q4'

In [30]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp=pd.read_excel('gdplev.xls',skiprows=7)
    
    gdp=gdp[['Unnamed: 4','Unnamed: 5']]
    
    quarters=[]
    gdp.columns=['Quarter','GDP']
    
    
    start=get_recession_start()
    end=get_recession_end()
    
    gdp_start=gdp[gdp['Quarter']==start].index.tolist()[0]
    gdp_end=gdp[gdp['Quarter']==end].index.tolist()[0]
    
    recessiongdp=gdp.iloc[gdp_start:gdp_end+1]
    
    
            
    
    return recessiongdp[recessiongdp['GDP']==np.min(recessiongdp['GDP'])]['Quarter'].tolist()[0]
get_recession_bottom()

'2009q2'

In [14]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    housing=pd.read_csv('City_Zhvi_AllHomes.csv')
    
    housing=housing.drop(housing.columns[[0]+list(range(3,51))],axis=1)
    
    housing=housing.reset_index()
    
    hdata=housing[['RegionName','State']]
    #hdata=hdata.reset_index()
    
    for year in range(2000,2016):
        hdata[str(year)+'q1']=housing[[str(year)+'-01',str(year)+'-02',str(year)+'-03']].mean(axis=1)
        hdata[str(year)+'q2']=housing[[str(year)+'-04',str(year)+'-05',str(year)+'-06']].mean(axis=1)
        hdata[str(year)+'q3']=housing[[str(year)+'-07',str(year)+'-08',str(year)+'-09']].mean(axis=1)
        hdata[str(year)+'q4']=housing[[str(year)+'-10',str(year)+'-11',str(year)+'-12']].mean(axis=1)
    year=2016
    hdata[str(year)+'q1']=housing[[str(year)+'-01',str(year)+'-02',str(year)+'-03']].mean(axis=1)
    hdata[str(year)+'q2']=housing[[str(year)+'-04',str(year)+'-05',str(year)+'-06']].mean(axis=1)
    hdata[str(year)+'q3']=housing[[str(year)+'-07',str(year)+'-08']].mean(axis=1)
        
    hdata=hdata.replace({'State':states})
    
    hdata=hdata.set_index(["State","RegionName"])
    
    return hdata


    

#convert_housing_data_to_quarters()

In [90]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    
    
    
    start=get_recession_start()
    bottom=get_recession_bottom()
    
    
    
    
    housing=convert_housing_data_to_quarters()
    housing=housing.reset_index()
    start = housing.columns[housing.columns.get_loc(start) -1]
    
    
    housing['ratio']=housing[bottom]-housing[start]
    
    housing=housing[['State','RegionName',start,bottom,'ratio']]
    
    housing=housing.reset_index()
    
    university_towns=get_list_of_university_towns()
    
    university_towns=university_towns.reset_index()
    
    
    
    
    
        
    #university_towns=university_towns.set_index(['State','RegionName'])
        
    #uprice=pd.merge(university_towns,hdf,left_on=['State','RegionName'],right_on=['State','RegionName'])
    
    #uprice=hdf[(hdf['State'].isin(university_towns['State'])) & (hdf['RegionName'].isin(university_towns['RegionName']))]
    
    
    
    uprice=pd.merge(housing,university_towns,how='inner',on=['State','RegionName'])
    
    uprice['uni']=True
    
    hdata=pd.merge(housing,uprice,how='left',on=['State','RegionName',start,bottom,'ratio'])
    
    hdata['uni']=hdata['uni'].fillna(False)
    
    uni=hdata[hdata['uni']==True]
    
    nonu=hdata[hdata['uni']==False]
    
    #nonuprice=hdf[(hdf['State'].isin(university_towns['State'])) & ~(hdf['RegionName'].isin(university_towns['RegionName']))]
    
    '''print(len(housing))
    
    print(len(uni))
    
    print(len(university_towns))
    
    print(len(nonu))'''
    
    
    t,p=ttest_ind(uni['ratio'].dropna(),nonu['ratio'].dropna())
    
    if p<0.01:
        different=True
    else:
        different=False
        
    if uni['ratio'].mean()>nonu['ratio'].mean():
        better='university town'
    else:
        better='non-university town'
    
    return (different,p,better)
run_ttest()




/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

10730
269
517
10461


(True, 0.002099659657952052, 'university town')